In [ ]:
import PIL.Image
from PIL import Image, ImageTk
from typing import Tuple
from tkinter import *
import cv2


In [7]:
from main import *
%reload_ext main

scale = 1
stride = 4
margin = 40

video = cv2.VideoCapture("videos/billiard_3_240.mp4")

_, first_frame = video.read()
first_frame = preprocess_frame(first_frame, scale=scale)
while True:
    selected_box = SelectBoxWindow.show_and_get_box(first_frame)
    if selected_box[-1] != 0 and selected_box[-2] != 0:
        break
print(selected_box)
# tracker = ObjectTracker(first_frame, selected_box,
#                         stride=stride, margin=margin)
# show_tracking_animation(video, tracker, scale=scale)


(720, 1280, 3)
(324, 197, 38, 36)


In [11]:
import pytictoc
from main import *
from matplotlib import pyplot as plt
%reload_ext main

timer = pytictoc.TicToc()

scale = 1
stride = 4
margin = 40

video = cv2.VideoCapture("videos/billiard_3.mp4")



def get_keypoints(img: np.ndarray, nfeatures=40, nOctaveLayers: int = 3, sigma: float = 1.5):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create(nfeatures, nOctaveLayers, sigma=sigma)
    kp, dsc = sift.detectAndCompute(gray, None)
    return kp, dsc, gray


def center_of_mass_match(img1: np.ndarray, img2: np.ndarray, cv2_norm: int = cv2.NORM_L1, nfeatures=40, nOctaveLayers: int = 3, sigma: float = 1.5, def_yx=(-1, -1)):
    kp1, dsc1, gray1 = get_keypoints(img1, nfeatures, nOctaveLayers, sigma)
    kp2, dsc2, gray2 = get_keypoints(img2, nfeatures, nOctaveLayers, sigma)
    bf = cv2.BFMatcher(cv2_norm, crossCheck=True)
    matches = bf.match(dsc1, dsc2)
    indexes = np.asarray([kp2[m.trainIdx].pt for m in matches], dtype='int')
    if len(indexes):
        x, y = np.mean(indexes, axis=0, dtype='int')
        return y, x

    # Not match found
    return def_yx


_, img = video.read()
margin = 10
y, x, w, h = (209, 760, 28, 29)
y, x, w, h = (324, 197, 38, 36)
height, width, _ = img.shape

dy, dx = -1, -1
part1 = img[y:y+h, x:x+w]
parts = []
for i in range(820):
    _, nimg = video.read()


    part2 = nimg[y-margin:y+h+margin, x-margin:x+w+margin]
    
    cy, cx = center_of_mass_match(part1, part2, nfeatures=30, nOctaveLayers=3, sigma=1.0, def_yx=(dy, dx))
    dy, dx = cy, cx

    _y, _x = (cy + dy) // 2 + y - part2.shape[0] // 2, (cx + dx) // 2 + x - part2.shape[1] // 2
    y, x = min(max(0, _y), height - h), min(max(0, _x), width - w)

    frame = cv2.rectangle(np.copy(nimg), (x, y), (x+w, y+h), (0, 0, 255), thickness=1)
    # Display the resulting frame
    cv2.imshow("Frame", frame)

    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord("q"):
        cv2.destroyAllWindows()
        break
    # dy, dx = cy, cx
    img = nimg
    
cv2.destroyAllWindows()


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [14]:
import numpy as np
import cv2 as cv

img = cv.imread(r"G:\Studia\AOC\zipped\airboard_1\240\airboard_1\00001.jpg")
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
sift = cv.SIFT_create()
kp = sift.detect(gray, None)
img = cv.drawKeypoints(gray, kp, img)
cv.imwrite("sift_keypoints.jpg", img)

True